In [393]:
import pandas as pd
import sqlalchemy 
import sql_functions as sf

In [359]:
# load jobs & skill data

schema = 'capstone_datacvpro'

jobs_20 = sf.get_dataframe(f' SELECT * FROM {schema}.analysts_20')
jobs_current = sf.get_dataframe(f' SELECT * FROM {schema}.jobs_current_skills_timeline')
skills_20 = sf.get_dataframe(f' SELECT * FROM {schema}.skills_20')

In [360]:
# drop skills with low count
df_skills_top = skills_20.query('count >= 40').copy()
df_skills_top.head()

,skill,count,type
0,Analytics,3385,Hard Skill
1,Management,1949,Soft Skill
2,Communications,1899,Soft Skill
3,SQL (Programming Language),1780,Hard Skill
4,Tooling,1769,Hard Skill


In [361]:
# drop entries that are not really skills

df_skills_top= df_skills_top.drop([0, 1, 4, 7, 9, 12, 13, 16, 20, 23, 44, 54, 57, 104]).reset_index(drop=True)
# dropped skills: analytics(0), management(1), tooling(4), operations(7), disabilities(9), levelling(12),  equalization(13), activism(20), industrialization(16), job descriptions(23), additives (44), governance (54), Medic(57)(used in context of medical insurance as benefit), Hostile Work Environment(104) 

# delete count column (shows count of every occurence of skill, also multiple times per description)
df_skills_top.drop(['count'], axis=1, inplace=True)

In [362]:
# remove everything in () so skills can be found in description

# split at ( and save in new column
df_skills_top['skill_clean'] = df_skills_top['skill'].str.split('(').str[0]
df_skills_top.head()

,skill,type,skill_clean
0,Communications,Soft Skill,Communications
1,SQL (Programming Language),Hard Skill,SQL
2,Positivity,Soft Skill,Positivity
3,Maintainability,Hard Skill,Maintainability
4,Collaboration,Soft Skill,Collaboration


In [363]:
# convert to lower case

df_skills_top['skill_clean'] = df_skills_top['skill_clean'].apply(lambda x: x.lower())

# change to singular
df_skills_top['skill_clean'][0] = 'communication'
df_skills_top['skill_clean'][7] = 'presentation'

df_skills_top.head(10)

,skill,type,skill_clean
0,Communications,Soft Skill,communication
1,SQL (Programming Language),Hard Skill,sql
2,Positivity,Soft Skill,positivity
3,Maintainability,Hard Skill,maintainability
4,Collaboration,Soft Skill,collaboration
5,Research,Soft Skill,research
6,Data Quality,Hard Skill,data quality
7,Presentations,Soft Skill,presentation
8,Statistics,Hard Skill,statistics
9,Dashboard,Hard Skill,dashboard


In [364]:
# save cleaned skills in list

skill_clean = df_skills_top.skill_clean.to_list()

In [365]:
# create copy of jobs_20 to modify

jobs_20_skills = jobs_20.copy()

In [366]:
# count each skill only once per job description to count how many job postings require specific skill

# create new columns for each skill that show 1 if skill is mentioned in job_description
for skill in skill_clean:
    jobs_20_skills[skill] = jobs_20_skills['job_description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0).copy()

jobs_20_skills.head()

/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/1631288791.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_20_skills[skill] = jobs_20_skills['job_description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0).copy()
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/1631288791.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_20_skills[skill] = jobs_20_skills['job_description'].apply(lambda x: 1 if skill.lower() in x.lower() else 0).copy()
/var/folders/sz/_wd1s7ln57l2hztzqprpx_

,job_title,salary_estimate,job_description,company_name,location,industry,communication,sql,positivity,maintainability,...,financial data management,chartered financial analyst,systems implementations,imaging,mobile app,employee assistance programs,modulation,scrum,medicaid,constructability
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,"New York, NY",Social Assistance,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,"New York, NY",Health Care Services & Hospitals,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,Squarespace\n3.4,"New York, NY",Internet,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,"New York, NY",IT Services,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,"New York, NY",Sports & Recreation,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [367]:
# create df with only skills to sum up the columns

jobs_20_skills = jobs_20_skills.drop(columns=['job_title', 'salary_estimate', 'job_description', 'company_name', 'location', 'industry'])

In [368]:
# create df with the columns skill and count, where skills are summed up by column

sum_skills = []

for skill_clean in jobs_20_skills.columns:
    sum = jobs_20_skills[skill_clean].sum()
    sum_skills.append({'skill_clean': skill_clean, 'count' : sum})

skills_count_clean = pd.DataFrame(sum_skills)
skills_count_clean.head()

,skill_clean,count
0,communication,1154
1,sql,990
2,positivity,5
3,maintainability,4
4,collaboration,245


In [369]:
# add type by merging with df_skills_top
# can be merged using index

skills_count = pd.merge(skills_count_clean, df_skills_top, left_index=True, right_index=True)
skills_count

,skill_clean_x,count,skill,type,skill_clean_y
0,communication,1154,Communications,Soft Skill,communication
1,sql,990,SQL (Programming Language),Hard Skill,sql
2,positivity,5,Positivity,Soft Skill,positivity
3,maintainability,4,Maintainability,Hard Skill,maintainability
4,collaboration,245,Collaboration,Soft Skill,collaboration
...,...,...,...,...,...
523,employee assistance programs,1,Employee Assistance Programs,Hard Skill,employee assistance programs
524,modulation,0,Modulation,Hard Skill,modulation
525,scrum,27,Scrum (Software Development),Hard Skill,scrum
526,medicaid,25,Medicaid,Hard Skill,medicaid


In [370]:
# delete duplicated column
skills_count.drop(['skill_clean_x'], axis=1, inplace=True)

# rename
skills_count.rename(columns={'skill_clean_y' : 'skill_clean'}, inplace=True)
skills_count.head(1)

,count,skill,type,skill_clean
0,1154,Communications,Soft Skill,communication


In [371]:
# rearrange column order
new_column_order = ['skill', 'skill_clean', 'count', 'type']

# reassign dataframe with new column order
skills_count = skills_count[new_column_order]

# sort values by count
skills_count = skills_count.sort_values('count', ascending=False)

skills_count

,skill,skill_clean,count,type
16,E (Programming Language),e,2248,Hard Skill
37,R (Programming Language),r,2233,Hard Skill
154,M (Programming Language),m,2110,Hard Skill
194,C (Programming Language),c,1639,Hard Skill
114,B (Programming Language),b,1500,Hard Skill
...,...,...,...,...
415,Oracle SQL Developer,oracle sql developer,0,Hard Skill
222,Economic Statistics,economic statistics,0,Hard Skill
220,JIRA Studio,jira studio,0,Hard Skill
418,Enterprise Report Management (ERM),enterprise report management,0,Hard Skill


In [372]:
# convert job_description to lower

jobs_20['job_description'] = jobs_20['job_description'].str.lower()

In [373]:
# Programming languages like R are counted in each word that contains an R
# to avoid that we extract standalone characters that are programming languages r,e,m,c,b

def has_standalone_r_e_m_c_b(text):
    words = text.split()
    return {'R': 'r' in words, 'E': 'e' in words, 'M': 'm' in words, 'C': 'c' in words, 'B': 'b' in words}

# apply the function to the "job_description" column and expand the result into separate columns
jobs_20[['R', 'E', 'M', 'C', 'B']] = jobs_20['job_description'].apply(has_standalone_r_e_m_c_b).apply(pd.Series)

# counting based on boolean mask
jobs_20[jobs_20["R"]]
print(len(jobs_20[jobs_20["R"]]), len(jobs_20[jobs_20["E"]]), len(jobs_20[jobs_20["M"]]), len(jobs_20[jobs_20["C"]]), len(jobs_20[jobs_20["B"]]))

# making a small table 
jobs_20[["R", "E", "M", "C", "B"]].sum(axis=0).sort_values(ascending = False)

158 7 1 15 11


R    158
C     15
B     11
E      7
M      1
dtype: int64

In [374]:
skills_count.head(6)

,skill,skill_clean,count,type
16,E (Programming Language),e,2248,Hard Skill
37,R (Programming Language),r,2233,Hard Skill
154,M (Programming Language),m,2110,Hard Skill
194,C (Programming Language),c,1639,Hard Skill
114,B (Programming Language),b,1500,Hard Skill
0,Communications,communication,1154,Soft Skill


In [375]:
# replace count of r,e,m,c,b in skills_count table

skills_count['count'][16] = 7
skills_count['count'][37] = 158
skills_count['count'][154] = 1
skills_count['count'][194] = 15
skills_count['count'][114] = 11

/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/4012136076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_count['count'][16] = 7
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/4012136076.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_count['count'][37] = 158
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/4012136076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [376]:
# rename to delete ' ' & change to upper case to merge with current list later

skills_count['skill_clean'][16] = 'E'
skills_count['skill_clean'][37] = 'R'
skills_count['skill_clean'][154] = 'M'
skills_count['skill_clean'][194] = 'C'
skills_count['skill_clean'][114] = 'B'

/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/3715030905.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_count['skill_clean'][16] = 'E'
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/3715030905.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skills_count['skill_clean'][37] = 'R'
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/3715030905.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [377]:
skills_count = skills_count.sort_values('count', ascending=False).reset_index(drop=True)
skills_count.head(10)

,skill,skill_clean,count,type
0,Communications,communication,1154,Soft Skill
1,SQL (Programming Language),sql,990,Hard Skill
2,Data Analysis,data analysis,717,Hard Skill
3,Computer Science,computer science,622,Hard Skill
4,Statistics,statistics,563,Hard Skill
5,Research,research,559,Soft Skill
6,Dashboard,dashboard,533,Hard Skill
7,Visualization,visualization,517,Hard Skill
8,Writing,writing,452,Soft Skill
9,Presentations,presentation,439,Soft Skill


In [378]:
skills_count.query('count == 0')

,skill,skill_clean,count,type
396,Process Automation Systems,process automation systems,0,Hard Skill
397,Lean Product Development,lean product development,0,Hard Skill
398,Physician Data Query,physician data query,0,Hard Skill
399,IText (Free PDF Software),itext,0,Hard Skill
400,Life Cycle Assessment,life cycle assessment,0,Hard Skill
...,...,...,...,...
523,Reservations,reservations,0,Soft Skill
524,Consumables,consumables,0,Hard Skill
525,Transferable Skills Analysis,transferable skills analysis,0,Hard Skill
526,Data Mapper Patterns,data mapper patterns,0,Hard Skill


In [379]:
jobs_20['job_description'].count()

2253

In [380]:
# add column with %

skills_count['frequency_%'] = skills_count['count'].apply(lambda x: x/(jobs_20['job_description'].count())*100).round(2)
skills_count = skills_count.sort_values('frequency_%', ascending=False).reset_index(drop=True)
skills_count.head()

,skill,skill_clean,count,type,frequency_%
0,Communications,communication,1154,Soft Skill,51.22
1,SQL (Programming Language),sql,990,Hard Skill,43.94
2,Data Analysis,data analysis,717,Hard Skill,31.82
3,Computer Science,computer science,622,Hard Skill,27.61
4,Statistics,statistics,563,Hard Skill,24.99


In [381]:
df = skills_count[skills_count['skill'].str.contains("statist", case=False)] 
df

,skill,skill_clean,count,type,frequency_%
4,Statistics,statistics,563,Hard Skill,24.99
82,Statistical Packages,statistical packages,97,Hard Skill,4.31
112,Statistical Software,statistical software,63,Hard Skill,2.80
158,Statistical Methods,statistical methods,45,Hard Skill,2.00
204,Statistical Modeling,statistical modeling,32,Hard Skill,1.42
213,SPSS (Statistical Software),spss,31,Hard Skill,1.38
224,Biostatistics,biostatistics,29,Hard Skill,1.29
402,Engineering Statistics,engineering statistics,0,Hard Skill,0.00
416,SAS Statistical Analysis,sas statistical analysis,0,Hard Skill,0.00
450,Mathematical Statistics,mathematical statistics,0,Hard Skill,0.00


In [159]:
# load to database

from dotenv import load_dotenv
load_dotenv()

# write dataset into database

# Import get_engine from sql_functions.py. You will need to restart your kernel and rerun at this point since we changed the module since we first imported it.
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

import psycopg2

table_name = 'skills_count_20'
schema = 'capstone_datacvpro'

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        skills_count.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The skills_count_20 table was imported successfully.


In [383]:
# load data of current dataset and merge into one dataset with skill distribution 2020 & current

skills_current = sf.get_dataframe(f' SELECT * FROM {schema}.skill_count_current')

skills_current.head()

,skill_clean,count_current,frequency_%_current
0,communication,16024,50.04
1,data analysis,13207,41.24
2,visualization,11400,35.60
3,sql,11117,34.72
4,dashboard,9736,30.40


In [384]:
# merge both skill tables

skill_count_merge = pd.merge(skills_count, skills_current, on='skill_clean')
skill_count_merge.head()

,skill,skill_clean,count,type,frequency_%,count_current,frequency_%_current
0,Communications,communication,1154,Soft Skill,51.22,16024,50.04
1,SQL (Programming Language),sql,990,Hard Skill,43.94,11117,34.72
2,Data Analysis,data analysis,717,Hard Skill,31.82,13207,41.24
3,Computer Science,computer science,622,Hard Skill,27.61,8160,25.48
4,Statistics,statistics,563,Hard Skill,24.99,8234,25.71


In [385]:
skill_count_merge.rename(columns= {'count' : 'count_20', 'frequency_%' : 'frequency_%_20'}, inplace=True)

# rearrange column order
new_column_order = ['skill', 'skill_clean', 'type', 'count_20', 'frequency_%_20', 'count_current', 'frequency_%_current']

# reassign dataframe with new column order
skill_count_merge = skill_count_merge[new_column_order]

skill_count_merge.sort_values('count_20', ascending=False).reset_index(drop=True)

skill_count_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   skill                528 non-null    object 
 1   skill_clean          528 non-null    object 
 2   type                 528 non-null    object 
 3   count_20             528 non-null    int64  
 4   frequency_%_20       528 non-null    float64
 5   count_current        528 non-null    int64  
 6   frequency_%_current  528 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 29.0+ KB


Clean up Top 50 skill list manually:
- summarize:
    - writing(8) + written commun.(24)
    - communications(0) + verbal communic(66) + interpersonal commun(352)
    - statistics(4) + (82,112,158,204,213,224,402,416,450,523)
    - visualization(7) + (16)
    - presentation(9) + (261, 437)
- delete (music) from programming (14)
- delete:
    - source, dataset, track, bus. requirements, acting (18,19,25,32,46)
    - delete all rows that were summarized

In [386]:
skill_count_merge['count_20'][8] = skill_count_merge['count_20'][8] + skill_count_merge['count_20'][24]
skill_count_merge['count_current'][8] = skill_count_merge['count_current'][8] + skill_count_merge['count_current'][24]

skill_count_merge['count_20'][0] = skill_count_merge['count_20'][0] + skill_count_merge['count_20'].iloc[[66, 352]].sum()
skill_count_merge['count_current'][0] = skill_count_merge['count_current'][0] + skill_count_merge['count_current'].iloc[[66, 352]].sum()

skill_count_merge['count_20'][4] = skill_count_merge['count_20'][4] + skill_count_merge['count_20'].iloc[[82,112,158,204,213,224,402,416,450,523]].sum()
skill_count_merge['count_current'][4] = skill_count_merge['count_current'][4] + skill_count_merge['count_current'].iloc[[82,112,158,204,213,224,402,416,450,523]].sum()

skill_count_merge['count_20'][7] = skill_count_merge['count_20'][7] + skill_count_merge['count_20'][16]
skill_count_merge['count_current'][7] = skill_count_merge['count_current'][7] + skill_count_merge['count_current'][16]

skill_count_merge['count_20'][9] = skill_count_merge['count_20'][9] + skill_count_merge['count_20'].iloc[[261, 437]].sum()
skill_count_merge['count_current'][9] = skill_count_merge['count_current'][9] + skill_count_merge['count_current'].iloc[[261,437]].sum()

/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/2435283833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_count_merge['count_20'][0] = skill_count_merge['count_20'][0] + skill_count_merge['count_20'].iloc[[66, 352]].sum()
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/2435283833.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_count_merge['count_current'][0] = skill_count_merge['count_current'][0] + skill_count_merge['count_current'].iloc[[66, 352]].sum()
/var/folders/sz/_wd1s7ln57l2hztzqprpx_4w0000gn/T/ipykernel_1165/2435283833.py:7: SettingWithCopyWarning: 
A value

In [387]:
skill_count_merge['skill'][14] = 'Programming'
skill_count_merge.iloc[14]

skill                   Programming
skill_clean            programming 
type                     Hard Skill
count_20                        382
frequency_%_20                16.96
count_current                  4309
frequency_%_current           13.46
Name: 14, dtype: object

In [388]:
skill_count_merge.drop([24,66,352,82,112,158,204,213,224,402,416,450,523,16,261,437,18,19,25,32,46], inplace=True)

In [389]:
skill_count_merge.head(1)

,skill,skill_clean,type,count_20,frequency_%_20,count_current,frequency_%_current
0,Communications,communication,Soft Skill,1285,51.22,18048,50.04


In [398]:
# rerun frequency code to calculate correct %

skill_count_merge['frequency_%_20'] = skill_count_merge['count_20'].apply(lambda x: x/(jobs_20['job_description'].count())*100).round(2)
skill_count_merge['frequency_%_current'] = skill_count_merge['count_current'].apply(lambda x: x/(jobs_current['index'].count())*100).round(2)

skill_count_merge = skill_count_merge.sort_values('frequency_%_20', ascending=False).reset_index(drop=True)
skill_count_merge.head()

,skill,skill_clean,type,count_20,frequency_%_20,count_current,frequency_%_current
0,Communications,communication,Soft Skill,1285,57.04,18048,56.36
1,SQL (Programming Language),sql,Hard Skill,990,43.94,11117,34.72
2,Statistics,statistics,Hard Skill,860,38.17,11580,36.16
3,Visualization,visualization,Hard Skill,856,37.99,18887,58.98
4,Writing,writing,Soft Skill,728,32.31,8131,25.39


In [399]:
# load to database

from dotenv import load_dotenv
load_dotenv()

# write dataset into database

# Import get_engine from sql_functions.py. You will need to restart your kernel and rerun at this point since we changed the module since we first imported it.
from sql_functions import get_engine
# create a variable called engine using the get_engine function
engine = get_engine()

import psycopg2

table_name = 'skills_count_total'
schema = 'capstone_datacvpro'

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        skill_count_merge.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

The skills_count_total table was imported successfully.
